In [125]:
from datetime import date, timedelta, datetime
import yfinance as yf

In [93]:
import pandas as pd

df = pd.read_csv (r'./client_value.csv')
print (df)

   past_value  current_value
0           0            100
Empty DataFrame
Columns: [past_value, current_value, time]
Index: []


In [94]:
def update_client_value(df, new_value): 
    past_value=df.iloc[0][1]
    df.iloc[0][0] = past_value
    df.iloc[0][1] = new_value   
    return df

In [ ]:
df = update_client_value(df,200)

In [114]:
df[['past_value','current_value']].to_csv('client_value.csv',index=False)

# Update Version 2 With Time

In [115]:
df_time = pd.read_csv (r'./client_valuev2.csv')
print (df_time)

Empty DataFrame
Columns: [past_value, current_value, time]
Index: []


In [116]:
df2 = {'past_value': 0, 'current_value': 500, 'time': date.today()}
df_time = df_time.append(df2, ignore_index = True)
df_time

,past_value,current_value,time
0,0,500,2022-05-24


In [117]:
def update_client_valuev2(df, new_value): 
    past_value = df.iloc[-1][1]
    df2 = {'past_value': past_value, 'current_value': new_value, 'time': datetime.today()}   
    df = df.append(df2, ignore_index = True)
    return df

In [122]:
df_time = update_client_valuev2(df_time,9000)

In [123]:
df_time

,past_value,current_value,time
0,0,500,2022-05-24
1,500,5000,2022-05-24 17:34:06.964684
2,5000,9000,2022-05-24 17:44:56.708835


In [121]:
df_time[['past_value','current_value','time']].to_csv('client_valuev2.csv',index=False)

# Portfolio

In [137]:
df_portfolio = pd.read_csv (r'./portfolio.csv')

In [138]:
df_portfolio

,coin,percentage,spent
0,BTC-USD,0.2,10
1,ETH-USD,0.1,15


In [139]:
def update_portfolio(df, value, signal, coin):

    df_coin = yf.download(coin,
                      start=date.today(), 
                      progress=False,
                      interval='1m'
    )
    if signal == 'buy':
            if coin not in df['coin'].tolist():
                                p= float(value/df_coin[df_coin.index == df_coin.index.min()]['Open'])
                                df2 = {'coin': coin, 'percentage':p, 'spent':value}
                                df = df.append(df2, ignore_index = True)

            
            else:

                for i, coin_name in enumerate(df.coin):
                    if coin_name==coin:
                        df['percentage'][i] = df['percentage'][i] + (value/df_coin[df_coin.index == df_coin.index.min()]['Open'])
                        df['spent'][i] = df['spent'][i] + value

            
    if signal == 'sell': 
        for i, coin_name in enumerate(df.coin):
                    if coin_name==coin:
                        if float(df['percentage'][i] - (value/df_coin[df_coin.index == df_coin.index.min()]['Open'])) < 0: return 'error'
                        df['percentage'][i] = df['percentage'][i] - (value/df_coin[df_coin.index == df_coin.index.min()]['Open'])
                        df['spent'][i] = df['spent'][i] - value 
        if coin not in df['coin'].tolist():
            print('error: coin not found')
    return df

In [140]:
df_portfolio = update_portfolio(df_portfolio, 100, signal='buy', coin='BTC-USD')

C:\Users\Admin\AppData\Local\Temp/ipykernel_23924/1955119821.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percentage'][i] = df['percentage'][i] + (value/df_coin[df_coin.index == df_coin.index.min()]['Open'])
C:\Users\Admin\AppData\Local\Temp/ipykernel_23924/1955119821.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['spent'][i] = df['spent'][i] + value


In [141]:
df_portfolio

,coin,percentage,spent
0,BTC-USD,0.203426,110
1,ETH-USD,0.100000,15


In [142]:
df_portfolio[['coin','percentage','spent']].to_csv('portfolio.csv',index=False)